### Tarea 9

Arturo González Bencomo  - 172906

## Ejercicio/Tarea 

Aprovecha la capacidad de Dask para realizar cómputo en paralelo para ajustar un modelo para predecir la proporción de propina de un viaje. Realiza búsqueda de hiperparámetros en grid con cross validation. Puedes usar funciones de scikit learn. Recuerda usar el decorador `delayed` para ejecutar en paralelo.

* ¿Qué tan rápido es buscar en paralelo comparado con una búsqueda secuencial en python?

Haz lo mismo que arriba, pero utilizando la biblioteca Dask-ML http://dask-ml.readthedocs.io/en/latest/ 

* ¿Cómo se comparan los tiempos de ejecución de tu búsqueda con la de Dask ML?

**Bonus**

Haz lo mismo utilizando Spark ML

* ¿Cómo se comparan los tiempos de ejecución de Spark vs Dask?

Usa los datos en s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv

* ¿Cambia alguno de los resultados anteriores?

### Cargamos dataset de viajes 

In [50]:
from dask import dataframe

#Leemos el archivo de csv
trips_df = dataframe.read_csv("/data/trips.csv")

#Eliminamos los viajes en donde no haya cuota 
trips_df = trips_df.loc[trips_df["fare_amount"]>0]
trips_df.head()

,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00


### Proporcion de propina

In [52]:
trips_df["proporcion_propina"] = trips_df.loc[:,["fare_amount","tip_amount"]].apply(lambda x: x.tip_amount/x.fare_amount,axis=1)
trips_df.head()

/usr/local/lib/python3.5/site-packages/dask/dataframe/core.py:2857: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


,car_type,fare_amount,passenger_count,taxi_id,tip_amount,tpep_dropoff_datetime,tpep_pickup_datetime,trip_distance,proporcion_propina
0,A,22.0,1,1,4.60,2015-01-03 01:37:02,2015-01-03 01:17:32,6.90,0.209091
1,A,9.0,1,1,0.00,2015-01-05 23:35:02,2015-01-05 23:25:15,1.81,0.000000
2,A,7.5,1,1,1.00,2015-01-06 15:22:12,2015-01-06 15:11:45,0.96,0.133333
3,A,8.5,1,1,1.00,2015-01-08 08:31:23,2015-01-08 08:22:12,1.90,0.117647
4,A,7.5,1,1,1.66,2015-01-08 12:35:54,2015-01-08 12:26:26,1.00,0.221333


### Modelo en dask Secuencial

Probamos con un modelo de regresion lineal simple con valores de default. 
Seleccionamos ciertas variables

In [95]:
from dask_ml.linear_model import LinearRegression

#Instanciamos un objeto de linear regression
lr = LinearRegression()

#Separamos en entrenamiento y prueba
entrenamiento, prueba = trips_df.random_split([0.7, 0.3], random_state=3)

x_entrena, y_entrena = entrenamiento[["passenger_count","trip_distance", "passenger_count", "fare_amount"]], entrenamiento["proporcion_propina"] 
x_prueba, y_prueba = prueba[["passenger_count","trip_distance", "passenger_count", "fare_amount"]], prueba["proporcion_propina"]

#Entrenamos el modelo
lr.fit(x_entrena.values.compute(),y_entrena.values.compute())


LinearRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
         intercept_scaling=1.0, max_iter=100, multiclass='ovr', n_jobs=1,
         penalty='l2', random_state=None, solver='admm',
         solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False)

#### Obtenemos los coeficientes

In [67]:
print("Coeficientes "+ str(lr.coef_))
print("Intercepto " + str(lr.intercept_))

Coeficientes [-0.00152645  0.00061793 -0.00152645 -0.00024799]
Intercepto 0.13531497908596862


#### Calculamos el error

In [72]:
valores_predichos = lr.predict(x_prueba.values.compute())

error_cuadratico = (valores_predichos - y_prueba.values.compute())**2

mse = sum(error_cuadratico)/len(error_cuadratico)

mse

0.019192113100240275

### Hacemos un gridsearch secuencial con dask

In [102]:
from dask_ml.model_selection import GridSearchCV
import timeit

#Instanciamos otro objeto de linear regression 
lr_grid = LinearRegression()

#Definimos un grid de parametros
grid_parametros = [{'C': [2,12,22],
                'tol': [0.01, 0.05, 0.1]}]

#Construimos un grisearchcv de dask en donde probamos el mejor modelo de regresion linean con los parametros definidos en la grid
#Definimos un parametro de cross validation de 3 y un solo job para que no ejecute en paralelo y tomamos tiempo de ejecucion

inicio = timeit.default_timer()
dask_grid_search = GridSearchCV(lr_grid, param_grid=grid_parametros, n_jobs=1, cv=3)
dask_grid_search.fit(x_entrena.values.compute(),y_entrena.values.compute())
fin_secuencial = timeit.default_timer() - inicio

In [105]:
dask_grid_search.best_params_

{'C': 2, 'tol': 0.1}

In [106]:
valores_predichos = dask_grid_search.predict(x_prueba.values.compute())

error_cuadratico = (valores_predichos - y_prueba.values.compute())**2

mse = sum(error_cuadratico)/len(error_cuadratico)

mse

0.019188607892603703

### Modelo en dask paralelo

In [107]:
#Se incluye este codigo para ejecutar las instrucciones de dask en paralelo

from dask.distributed import Client
client = Client("scheduler:8786")

In [108]:
#Construimos un grisearchcv de dask en donde probamos el mejor modelo de regresion linean con los parametros definidos en la grid
#Definimos un parametro de cross validation de 3 y 4 jobs para que se ejecute en paralelo utilizando todos los procesadores de mi computadora
dask_grid_search = GridSearchCV(lr_grid, param_grid=grid_parametros, n_jobs=4, cv=3)
inicio = timeit.default_timer()
dask_grid_search.fit(x_entrena.values.compute(),y_entrena.values.compute())
fin_paralelo =  timeit.default_timer() - inicio

In [99]:
dask_grid_search.best_params_

{'C': 2, 'tol': 0.1}

#### Calcullamos el error de paralelo: 

In [98]:
valores_predichos = dask_grid_search.predict(x_prueba.values.compute())

error_cuadratico = (valores_predichos - y_prueba.values.compute())**2

mse = sum(error_cuadratico)/len(error_cuadratico)

mse

0.019188607892603703